In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from confluent_kafka import Producer, Consumer
import json

In [ ]:
# Kafka producer configuration
bootstrap_servers = 'pkc-lzvrd.us-west4.gcp.confluent.cloud'
sasl_username = 'GPTZSLOX4PMDS2ZK'
sasl_password = 'ojOwC3qCEzEBcfniST+ddVRZynkuw/xVbXQcVOBBAhmjd07Ips+Y4nFffiFg16RK'
topic_name = 'network-traffic'

spark = SparkSession.builder \
    .appName("network_traffic_monitoring") \
    .getOrCreate()

# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("subscribe",topic_name) \
  .option("kafka.bootstrap.servers", "pkc-lzvrd.us-west4.gcp.confluent.cloud:9092") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "PLAIN") \
  .option("kafka.sasl.username", sasl_username) \
  .option("kafka.sasl.password", sasl_password) \
  .load()

parsed_df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Total bytes
total_bytes = parsed_df \
    .withWatermark("timestamp", "5 minutes") \
    .groupBy(window("timestamp", "5 minutes", "1 minute")) \
    .agg(sum("bytes_sent").alias("total_bytes"))

# Average Bytes
average_bytes = parsed_df \
    .withWatermark("timestamp", "5 minutes") \
    .groupBy(window("timestamp", "5 minutes", "1 minute")) \
    .agg(avg("bytes_sent").alias("average_bytes"))# Kafka producer configuration

In [ ]:
# Produce processed data

processed_topic = "proccessed-traffic"

producer = Producer({
    'bootstrap.servers': f"{bootstrap_servers}:9092",
    'security.protocol':'SASL_SSL',
    'sasl.mechanism':'PLAIN',
    'sasl.username': sasl_username,
    'sasl.password':sasl_password
})
processed = {
    "average_bytes": average_bytes,
    "total_bytes": total_bytes
}
p_string = json.dumps(processed)

producer.produce(processed_topic,  key = "key", value = p_string.encode('utf-8'))